In [1]:
from sklearn.metrics import confusion_matrix, precision_score
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Dropout
from keras.models import Sequential
from keras.regularizers import l2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

C:\Users\praveen ji\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [24]:
featureDF_nw = pd.read_csv('nw_feature_woText_MSFT.csv')
featureDF_tw = pd.read_csv('tw_feature_woText_MSFT.csv')
featureFI = pd.read_csv('FIwithTargetwithFTT.csv')
bert_featureDF_nw = pd.read_csv('nw_BertVector.csv')
bert_featureDF_tw = pd.read_csv('tw_BertVector_MSFT.csv')

In [25]:
def getStartEndDateShape(dfz):
    print('---------------------------')   
    print('shape', dfz.shape) 
    try:
        startDate = str(min(dfz['date']))
        endDate = str(max(dfz['date']))
        print('startDate', startDate)
        print('endDate', endDate)

    except:
        print("No Date column")

In [26]:
getStartEndDateShape(featureDF_nw)
getStartEndDateShape(featureDF_tw)
getStartEndDateShape(featureFI)
getStartEndDateShape(bert_featureDF_nw)
getStartEndDateShape(bert_featureDF_tw)

---------------------------
shape (4020, 24)
startDate 2004/07/21
endDate 2019/03/28
---------------------------
shape (1426, 25)
startDate 2015/05/04
endDate 2019/04/04
---------------------------
shape (4010, 32)
startDate 2003/04/30
endDate 2019/04/03
---------------------------
shape (4020, 768)
No Date column
---------------------------
shape (1426, 768)
No Date column


# Formulation of overall dataset

In [27]:
nw_date = featureDF_nw.date.values.tolist()
tw_date = featureDF_tw.date.values.tolist()

In [28]:
bert_featureDF_nw['date'] = nw_date
bert_featureDF_tw['date'] = tw_date

In [29]:
result =pd.merge( featureFI,featureDF_nw, how='outer', on='date')
getStartEndDateShape(result)

---------------------------
shape (4749, 55)
startDate 2003/04/30
endDate 2019/04/03


In [30]:
result =pd.merge( result,featureDF_tw, how='outer', on='date')
getStartEndDateShape(result)

---------------------------
shape (5004, 79)
startDate 2003/04/30
endDate 2019/04/04


In [ ]:
#start only scaling engineered features

In [10]:
resultScaled =result.copy()

In [11]:
tdate= resultScaled.date.tolist()
ttarget = resultScaled.Target.tolist()
resultScaled.drop(['date','Target'],axis =1,inplace=True)

In [12]:
scale = StandardScaler()
resultScaled = pd.DataFrame(scale.fit_transform(resultScaled.values), columns=resultScaled.columns, index=resultScaled.index)

In [13]:
resultScaled['date'] = tdate
resultScaled['target'] = ttarget

In [15]:
resultScaled =pd.merge( resultScaled,bert_featureDF_nw, how='outer', on='date')
getStartEndDateShape(resultScaled)

---------------------------
shape (5004, 847)
startDate 2003/04/30
endDate 2019/04/04


In [16]:
resultScaled =pd.merge( resultScaled,bert_featureDF_tw, how='outer', on='date')
getStartEndDateShape(resultScaled)

---------------------------
shape (5004, 1615)
startDate 2003/04/30
endDate 2019/04/04


In [17]:
resultScaled.sort_values(by='date',inplace=True) 
resultScaled = resultScaled.ffill()
resultScaled = resultScaled.bfill()
resultScaled.fillna(0,inplace=True)

In [18]:
resultScaled.date = pd.to_datetime(resultScaled.date)

In [ ]:
#end only scaling engineered features

In [31]:
result =pd.merge( result,bert_featureDF_nw, how='outer', on='date')
getStartEndDateShape(result)

---------------------------
shape (5004, 847)
startDate 2003/04/30
endDate 2019/04/04


In [32]:
result =pd.merge( result,bert_featureDF_tw, how='outer', on='date')
getStartEndDateShape(result)

---------------------------
shape (5004, 1615)
startDate 2003/04/30
endDate 2019/04/04


In [33]:
result.sort_values(by='date',inplace=True) 

In [34]:
result = result.ffill()
result = result.bfill()
result.fillna(0,inplace=True)

In [35]:
result.date = pd.to_datetime(result.date)

In [ ]:
#start non Scaled Data Prep

In [36]:
resultNonScaledData = result.copy()

In [ ]:
#end non Scaled Data Prep

In [29]:
date= result.date.tolist()
target = result.Target.tolist()
result.drop(['date','Target'],axis =1,inplace=True)

In [32]:
scale = StandardScaler()
result = pd.DataFrame(scale.fit_transform(result.values), columns=result.columns, index=result.index)

In [33]:
result['date'] = date
result['target'] = target

In [34]:
# Save data with without scaling
result.to_csv('DFScaledNTFI_with_scaling.csv',index= False)

# Test and Train Split

In [36]:
train = result[result['date'] < pd.to_datetime('2018/06/01')]
test = result[result['date'] >= pd.to_datetime('2018/06/01')]

In [37]:
train.shape

(4696, 1615)

In [39]:
test.shape

(308, 1615)

In [40]:
train.to_csv('train_bert_tnfi.csv',index=False)
test.to_csv('test_bert_tnfi.csv',index=False)

In [ ]:
#start only scaling engineered features

In [19]:
train_s = resultScaled[resultScaled['date'] < pd.to_datetime('2018/06/01')]
test_s = resultScaled[resultScaled['date'] >= pd.to_datetime('2018/06/01')]

In [22]:
print(train_s.shape)
print(test_s.shape)

(4696, 1615)
(308, 1615)


In [23]:
train_s.to_csv('train_SEFnNSB_bert_tnfi.csv',index=False)
test_s.to_csv('test_SEFnNSB_bert_tnfi.csv',index=False)

In [ ]:
#end only scaling engineered features

In [ ]:
#start only non-scaled dataset

In [37]:
train_ns = resultNonScaledData[resultNonScaledData['date'] < pd.to_datetime('2018/06/01')]
test_ns = resultNonScaledData[resultNonScaledData['date'] >= pd.to_datetime('2018/06/01')]

In [38]:
print(train_ns.shape)
print(test_ns.shape)

(4696, 1615)
(308, 1615)


In [40]:
train_ns.to_csv('train_nS_bert_tnfi.csv',index=False)
train_ns.to_csv('test_nS_bert_tnfi.csv',index=False)

In [ ]:
#end only non-scaled dataset